# Excel_file_split

In [1]:
# Automates split of any multi-sheet Excel file given a length constraint
# Splits each sheet of Excel file multiple times based on constraints and exports each new data subset as Excel file

# Notes:
## Excel file name can be used instead of path if Excel file is located in same directory as notebook
## column name specifies which column is used when grouping rows for file split
### i.e. split may occur at row 88 instead of the max_split_value = 100 to keep rows with same column value together
## max_split_value specifies the desired maximum length of each Excel file after splitting. Default is 100.

# Example:
## Excel_file_split('EXCEL_FILE.xlsx', 'COLUMN_NAME', 120)

import pandas as pd


def Excel_file_split(file_name_or_path, column_name, max_split_value=100):
    
    ## FUNCTIONS
    
    # Function that gives the index at which to split the Excel file given a DataFrame df.
    # Can also be given a max_split value, but default set to 120.

    def RowSplit(df):

        # Empty list to store indices of potential split
        i_splits = []

        # Want to locate max index <= st 'COLUMN_NAME' @ i != 'COLUMN_NAME' @ i = i+1 (where a given row and the next row have different column names)
        for i in range(max_split_value):
            if df.loc[i,column_name] != df.loc[i+1,column_name]:
                i_splits.append(i+1)

        # Take maximum index from stored list
        max_i = max(i_splits)
        
        return max_i
        

    # Function that splits data based on RowSplit function output max_i and exports it to Excel before resetting index of the
    # remaining data

    def SplitSheet(df):

        # Determine where to split data
        max_i = RowSplit(df)

        # Split DataFrame at max_i and export as new Excel file
        data_1 = df.head(max_i)
        data_1.to_excel(str(sheet_names[i])+"_"+str(k)+".xlsx", index=False)

        # Rename remainder of data after split and reset index
        data_2 = df.loc[max_i:len(df)]
        data_2 = data_2.reset_index()
        del data_2['index']

        return data_2 

    
    # Read Excel file
    xl = pd.ExcelFile(file_name_or_path)

    # Save list of sheet names within Excel file
    sheet_names = xl.sheet_names

    # Parse through sheet_names - index 0 is just 'Notes'
    for i in range(len(sheet_names)):
        data = xl.parse(sheet_names[i])

        # Run code
        k = 0

        while len(data)>max_split_value:
            k = k + 1
            data = SplitSheet(data)
        
        # Export last DataFrame to Excel
        k = k + 1
        data.to_excel(str(sheet_names[i])+"_"+str(k)+".xlsx", index=False)


    return None

### Illustration of row splitting scheme with length constraint and specified column name using CSV file and Pandas DataFrame

In [2]:
import pandas as pd

# Load CSV file as Pandas DataFrame
car_sales = pd.read_csv("/Users/victoriauribe/Code/code-2022/Data/car_sales.csv")

# Sort data based on column 'max_power'
car_sales = car_sales.sort_values(by=['max_power'], ascending=True)

# Reset index of DataFrame based on sort
car_sales = car_sales.reset_index()
del car_sales['index']

# Determine where to split DataFrame based on length constraint and column name
def RowSplit(df, column_name, max_split_value):

        # Empty list to store indices of potential split
        i_splits = []

        # Want to locate max index <= st 'COLUMN_NAME' @ i != 'COLUMN_NAME' @ i = i+1 (where a given row and the next row have different column names)
        for i in range(max_split_value):
            if df.loc[i,column_name] != df.loc[i+1,column_name]:
                i_splits.append(i+1)

        # Take maximum index from stored list
        max_i = max(i_splits)
        
        return max_i

# Specify column_name as 'max_power' and set max_split_value = 110 for length constraint
max_i = RowSplit(car_sales, 'max_power', 110)
print(max_i)

105


#### Function RowSplit determines that DataFrame should be split at row 105 based on column 'max_power' and maximum_split_value = 110

In [3]:
car_sales.loc[104:110]

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
104,Toyota Innova 2.5 EV (Diesel) PS 7 Seater BS IV,2013,600000,95000.0,Diesel,Individual,Manual,Second Owner,12.99 kmpl,2494 CC,100.57 bhp,200Nm@ 1400-3400rpm,7.0
105,Toyota Innova 2.5 G (Diesel) 7 Seater,2014,700000,300000.0,Diesel,Individual,Manual,Second Owner,12.99 kmpl,2494 CC,100.6 bhp,200Nm@ 1400-3400rpm,7.0
106,Toyota Innova 2.5 VX (Diesel) 7 Seater,2015,1200000,61000.0,Diesel,Individual,Manual,First Owner,12.99 kmpl,2494 CC,100.6 bhp,200Nm@ 1200-3600rpm,7.0
107,Toyota Innova 2.5 VX (Diesel) 7 Seater,2013,750000,79328.0,Diesel,Trustmark Dealer,Manual,Second Owner,12.99 kmpl,2494 CC,100.6 bhp,200Nm@ 1200-3600rpm,7.0
108,Toyota Innova 2.5 VX (Diesel) 7 Seater,2013,750000,79328.0,Diesel,Trustmark Dealer,Manual,Second Owner,12.99 kmpl,2494 CC,100.6 bhp,200Nm@ 1200-3600rpm,7.0
109,Toyota Innova 2.5 ZX Diesel 7 Seater BSIII,2015,1030000,190000.0,Diesel,Individual,Manual,First Owner,12.99 kmpl,2494 CC,100.6 bhp,200Nm@ 1400-3400rpm,7.0
110,Toyota Innova 2.5 VX (Diesel) 7 Seater,2013,750000,79328.0,Diesel,Trustmark Dealer,Manual,Second Owner,12.99 kmpl,2494 CC,100.6 bhp,200Nm@ 1200-3600rpm,7.0


#### Here we see that splitting at the maximum_split_value of 110 would split the data within the same values of max_power, 100.6 bhp. Instead, we split at 105, so that the values of 100.6 bhp will be exported to the same Excel file.